In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

library_path = os.path.abspath("../library")
if library_path not in sys.path:
    sys.path.append(library_path)

In [3]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Disable the warning

import pickle
import numpy as np

from metrics import SvdMetricsCalculator, SvdTestMetricsCalculator
from rating import get_explicit_rating, get_implicit_rating_out_of_positive_ratings_df, split_matrix_csr, sanity_check_implicit_rating, sanity_check_explicit_split, sanity_check_explicit_matrix
from tuning import GridSearchSvdPP

# Feature selection

The only dataset that is necessary for our purposes is **review** dataset since:
- it contains the information about explicit ratings (the mean of the field **stars** for pairs of users and items, check the chapter **Feature engineering** for more details)
- it contains the information for implicit rating (check the chapter **Feature engineering** for more details)
- it already contains only those users who provided at least one review and those items that received at least one estimation

In [3]:
PATH = '../../../eda/dataset_samples/df_yelp_review_open_health_10.parquet'

There are two paths of working with the dataset that contains more than **900 000** rows

*The standard flow:*
1. Download dataset from Kaggle
2. Drop the reviews for closed businesses (`is_open == True`) and select the particular category of business (check chosen category in `/eda/yelp.ipynb`)
3. Drop the reviews were provided by users that assessed less than **threshold** items (check the threshold definition in `/eda/yelp.ipynb`) 
4. Use all the dataset in the purpose of analysis, training and testing

*The "sampled" flow* (provided in `/eda/yelp.ipynb`):
1. Download dataset from Kaggle
2. Drop the reviews for closed businesses (`is_open == True`) and select the particular category of business (check chosen category in `/eda/yelp.ipynb`)
3. Drop the reviews were provided by users that assessed less than **threshold** items (check the threshold definition in `/eda/yelp.ipynb`) 
4. Sample **10%** randomly
5. Check similarity of mathematical moments (`.describe()`) and dataset balancing (`.value_counts()`) with main dataset (initial YELP)

In the purpose of time-efficiency and due to the lack of power **the "sampled" flow** was chosen during EDA, and the sampled dataset was saved using the following path **`/eda/dataset_samples`**

Reviews' features:
- `review_id` | `user_id` | `business_id` - id of the review and foreign keys (one user can leave several reviews for one item)
- `stars` - **explicit rating** provided by user for the particular item in the particular moment
- `useful` | `funny` | `cool`  - user's flags about (presumably) review. We don't drop this feature since it's necessary to get any evidences that theory about the nature of the feature is right - check the **opportunity of usage them for implicit rating** 
- `text` - the content of review (can be useful for potential sentimental analysis)
- `date` - the timestamp of review

In [4]:
review_df = pd.read_parquet(PATH)
review_df

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,year,review_length
0,A8JR77dMCrnymyMhPQE7Qw,tL2pS5UOmN6aAOi3Z-qFGg,kEC675O6YwRH30ImVxBCCA,4,2017-02-16,I'm new to the area and stopped in here a few ...,0,0,0,2017,96
1,6Gd8tlhutYDMifKEzCvHyw,tL2pS5UOmN6aAOi3Z-qFGg,07F0eE_JHkH4Op0vpR_v4A,2,2013-05-09,I lived in this neighborhood a number of years...,2,0,0,2013,104
2,ZDMj4LnkO26QtM-CDtV94w,tL2pS5UOmN6aAOi3Z-qFGg,m-_IOYAreUyy_uyK9U3niQ,5,2013-02-03,It's been a few years since I've hiked this tr...,3,1,1,2013,109
3,8yfmu2iAagShVXJUGVTELw,tL2pS5UOmN6aAOi3Z-qFGg,6zry3kyGHiplbQ4rdqxbaQ,4,2013-07-17,This is a challenging hike to one of the most ...,0,0,0,2013,46
4,HaIBF7a1HjFT-gOIlmSIdQ,tL2pS5UOmN6aAOi3Z-qFGg,b3z314J6wktaNVblxumiug,5,2014-07-18,This is one if my favorite trails in the Redro...,0,0,0,2014,42
...,...,...,...,...,...,...,...,...,...,...,...
31736,JZJts6Y7gG5Uog6zyiXo5w,IqIpCfg0qDhIkaUJGKzlyw,KlLCJN_KUP9xFQBJYrhgVg,5,2012-10-21,pretty hospital great staff!! I had a great p...,0,0,0,2012,31
31737,Lp4ZHWDrDoXM2HHkZTh90g,IqIpCfg0qDhIkaUJGKzlyw,U-9uOCu4tG4idBAnMPmZTw,4,2012-08-15,I like this place because I live by it!! Mini ...,1,1,1,2012,57
31738,oQP82Wz-gIlV2qG53EacGA,IqIpCfg0qDhIkaUJGKzlyw,wghDrzcZ0VloAtaIZ7GEBg,4,2013-06-07,There's parking thank goodness. I hiked here f...,2,0,0,2013,41
31739,KZEEqRkPNDVTCZCBb93iYA,IqIpCfg0qDhIkaUJGKzlyw,4xkjmpgUNJdwQo8FKIYp6Q,3,2012-07-28,I bought a groupon yay at $13 and it was so wo...,5,1,1,2012,166


Reasons of feature dropping:
- since `useful | funny | cool` features describe the preferences of other users about this particular review, not an item, they can't be used for calculations of **explicit** and **implicit** ratings (our assumption is that these features are reactions that user can give to review that theoretically describes the user-to-user relations) 
- `review_id` won't be dropped, but will be used for indexing since it's unique field
- `text` of review won't be used for implicit or explicit ratings so this feature can be also dropped 

In [5]:
REMAINED_FEATURES = ['review_id', 'user_id', 'business_id', 'stars', 'date']

filtered_review_df = review_df[REMAINED_FEATURES]
filtered_review_df.set_index('review_id', inplace=True)
filtered_review_df

,user_id,business_id,stars,date
review_id,,,,
A8JR77dMCrnymyMhPQE7Qw,tL2pS5UOmN6aAOi3Z-qFGg,kEC675O6YwRH30ImVxBCCA,4,2017-02-16
6Gd8tlhutYDMifKEzCvHyw,tL2pS5UOmN6aAOi3Z-qFGg,07F0eE_JHkH4Op0vpR_v4A,2,2013-05-09
ZDMj4LnkO26QtM-CDtV94w,tL2pS5UOmN6aAOi3Z-qFGg,m-_IOYAreUyy_uyK9U3niQ,5,2013-02-03
8yfmu2iAagShVXJUGVTELw,tL2pS5UOmN6aAOi3Z-qFGg,6zry3kyGHiplbQ4rdqxbaQ,4,2013-07-17
HaIBF7a1HjFT-gOIlmSIdQ,tL2pS5UOmN6aAOi3Z-qFGg,b3z314J6wktaNVblxumiug,5,2014-07-18
...,...,...,...,...
JZJts6Y7gG5Uog6zyiXo5w,IqIpCfg0qDhIkaUJGKzlyw,KlLCJN_KUP9xFQBJYrhgVg,5,2012-10-21
Lp4ZHWDrDoXM2HHkZTh90g,IqIpCfg0qDhIkaUJGKzlyw,U-9uOCu4tG4idBAnMPmZTw,4,2012-08-15
oQP82Wz-gIlV2qG53EacGA,IqIpCfg0qDhIkaUJGKzlyw,wghDrzcZ0VloAtaIZ7GEBg,4,2013-06-07


# Explicit rating extracting

Convert the "date" column in the filtered dataset to UNIX timestamp (in milliseconds)

In [6]:
filtered_review_df["date"] = pd.to_datetime(filtered_review_df["date"]).astype(np.int64) // 10 ** 9
filtered_review_df

,user_id,business_id,stars,date
review_id,,,,
A8JR77dMCrnymyMhPQE7Qw,tL2pS5UOmN6aAOi3Z-qFGg,kEC675O6YwRH30ImVxBCCA,4,1487203200
6Gd8tlhutYDMifKEzCvHyw,tL2pS5UOmN6aAOi3Z-qFGg,07F0eE_JHkH4Op0vpR_v4A,2,1368057600
ZDMj4LnkO26QtM-CDtV94w,tL2pS5UOmN6aAOi3Z-qFGg,m-_IOYAreUyy_uyK9U3niQ,5,1359849600
8yfmu2iAagShVXJUGVTELw,tL2pS5UOmN6aAOi3Z-qFGg,6zry3kyGHiplbQ4rdqxbaQ,4,1374019200
HaIBF7a1HjFT-gOIlmSIdQ,tL2pS5UOmN6aAOi3Z-qFGg,b3z314J6wktaNVblxumiug,5,1405641600
...,...,...,...,...
JZJts6Y7gG5Uog6zyiXo5w,IqIpCfg0qDhIkaUJGKzlyw,KlLCJN_KUP9xFQBJYrhgVg,5,1350777600
Lp4ZHWDrDoXM2HHkZTh90g,IqIpCfg0qDhIkaUJGKzlyw,U-9uOCu4tG4idBAnMPmZTw,4,1344988800
oQP82Wz-gIlV2qG53EacGA,IqIpCfg0qDhIkaUJGKzlyw,wghDrzcZ0VloAtaIZ7GEBg,4,1370563200


Calculating the **explicit rating** for the filtered dataset. 

The output consists of two CSR matrices with identical structure: the first matrix contains **the mean review rating** given by user *u_i* to business *b_i*, and the second matrix stores **the timestamp of the latest review** at the same positions. 

Additionally, two utility dictionaries are provided, containing mappings **between IDs and matrix indices** (and vice versa).

In [7]:
explicit_ratings, last_dates, user_mapping, item_mapping = get_explicit_rating(filtered_review_df, "user_id",
                                                                               "business_id", "stars", "date")

explicit_ratings.toarray(), last_dates.toarray()

(array([[4., 2., 5., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 4., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[1487203200, 1368057600, 1359849600, ...,          0,          0,
                  0],
        [         0,          0,          0, ...,          0,          0,
                  0],
        [         0,          0,          0, ...,          0,          0,
                  0],
        ...,
        [         0,          0,          0, ...,          0,          0,
                  0],
        [         0,          0,          0, ..., 1486771200,          0,
                  0],
        [         0,          0,          0, ...,          0,          0,
                  0]]))

**Sanity check**:
* the amount of filled cells in the sparse matrices (`.nnz`) must be the same as **the number of unique pairs** of users and items
* the amount is **the same**

In [8]:
sanity_check_explicit_matrix(explicit_ratings=explicit_ratings, last_dates=last_dates, review_df=filtered_review_df)

,Source,Calculated metrics,Value
0,Explicit ratings matrix,Non-zero entries,31740
1,Last dates matrix,Non-zero entries,31740
2,Filtered review DataFrame,"Unique (user_id, business_id) pairs",31740


# Train / validation / test split

Define the divisions within the initial matrix (**test / validation / train** according to the documentation of split function)

In [9]:
DIVISIONS = [0.1, 0.2, 0.7]

Split matrix in proportions `0.1, 0.2, 0.7` for test, validation and train set in the following way:
1. Validate that the ratings and timestamps matrices have the same shape.
2. Ensure the sum of all ratios is equal to 1.0.
3. Initialize empty containers (data, indices, indptr) for each target split.
4. Precompute the ideal number of non-zero entries per split based on the total nnz and the given ratios.
5. For each row in the matrix:
    1. Extract non-zero ratings, timestamps, and their column indices.
    2. Sort the entries by timestamp in descending order (most recent first).
    3. Iterate through each ratio:
        * Compute how many elements to assign to the current split.
        * Use floor or round to balance the global target distribution.
        * For the last split, assign all remaining elements to ensure no data is lost.
        * Append the selected entries to the corresponding split containers.
6. Construct new CSR matrices for each split using the collected data.
7. Return the list of split CSR matrices.


In [10]:
test_matrix, validation_matrix, train_matrix = split_matrix_csr(explicit_ratings, last_dates, DIVISIONS)
train_matrix.toarray(), validation_matrix.toarray(), test_matrix.toarray()

(array([[0., 2., 5., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 4., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[4., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

**Sanity check** (verify that the explicit matrix has been correctly split into **train, validation, and test** subsets):
* The total number of interactions (nnz) in the splits matches the original explicit matrix.
* The proportions of data in each split (Train, Validation, Test) **almost** align with the intended ratios.
* No interactions are lost during the split.

In [13]:
sanity_check_explicit_split(train_matrix=train_matrix, validation_matrix=validation_matrix, test_matrix=test_matrix, explicit_matrix=explicit_ratings)

,Split,Number of interactions,Part of factual interactions
0,Train,22548,71.04%
1,Validation,6136,19.33%
2,Test,3056,9.63%
3,Explicit total,31740,100.0%
4,Factual total,31740,100%


# Implicit rating extraction for train dataset

**The threshold** for implicit ratings calculations (only positive ratings are considered and explicit ratings are `from 1 to 5`)

In [11]:
IMPLICIT_THRESHOLD = 4

Calculate the **implicit rating** in the following way:
* calculate the amount of reviews from `u_i` to `b_i` that have the number of starts is above `IMPLICIT_THRESHOLD`

Final artifact:
* dict in the format `{<user_id>: {<item_id>: <amount of positive reviews>} }`

In [13]:
implicit_ratings = get_implicit_rating_out_of_positive_ratings_df(df=filtered_review_df, user_field='user_id',item_field='business_id', rating_field='stars', implicit_threshold=IMPLICIT_THRESHOLD)

len(implicit_ratings.keys()), filtered_review_df['user_id'].nunique()

(1696, 1696)

**Sanity check** (verify the correctness of the `implicit_ratings` matrix creation from the filtered review DataFrame using a specified `IMPLICIT_THRESHOLD`):

Metrics:
* Total number of reviews in the original dataset that meet or exceed the implicit threshold.
* Confirms that all qualifying reviews were included in the final implicit ratings matrix.
* Number of distinct users present in the filtered dataset before conversion.
* Ensures that no user information was lost during transformation.
* Total businesses that were reviewed in the original dataset.
* Confirms that all relevant business interactions are retained.

Results: 
* All user and business counts match between the initial and processed datasets.
* The number of implicit ratings is equal to the number of qualifying reviews — indicating a correct threshold-based transformation.

In [16]:
sanity_check_implicit_rating(initial_df=filtered_review_df, implicit_ratings=implicit_ratings, implicit_threshold=IMPLICIT_THRESHOLD)

,Metric,Value
0,Number of reviews (stars >= threshold),22636
1,Number of reviews in implicit_ratings,22636
2,Unique users in initial reviews,1696
3,Unique users in implicit_ratings,1696
4,Unique businesses in initial reviews,8406
5,Unique businesses in implicit_ratings,8406


# Hyperparameters tuning

Define potential values of hyperparameters for the implementation of **SVD++** 

In [17]:
svd_pp_param_grid = {
    'n_factors': [20, 40, 70, 100, 150],    # Number of latent factors
    'n_epochs': [10, 20, 30, 50],           # Number of training iterations
    'lr_all': [0.002, 0.005, 0.007, 0.01],  # Learning rate
    'reg_all': [0.02, 0.05, 0.1, 0.2]       # Regularization strength
}

Using **train and validation** dataset conduct grid search based on **RMSE** metric and extract the best hyperparameters for the target metric on the validation dataset. 

Best hyperparameters (based on validation matrix):
* **learning rate**: 0.007
* **number of epochs**: 50
* **number of hidden factors**: 40
* **regularization term**: 0.2

Best RMSE: **1.111** (in average the model makes in 1.111 point in rating estimation)

In [18]:
grid_search_svd_pp = GridSearchSvdPP(train_matrix=train_matrix, val_matrix=validation_matrix,
                                     implicit_rating=implicit_ratings, user_mapping=user_mapping,
                                     item_mapping=item_mapping, param_grid=svd_pp_param_grid)

best_params, best_score, best_svdpp_model = grid_search_svd_pp.run()

print(f"Best params: {best_params}")
print(f"Best RMSE: {best_score}")

INFO:root:Try number: 1
INFO:root:Train with params: {'lr_all': 0.002, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.02}
INFO:root:Epoch: 1
INFO:root:Epoch: 2
INFO:root:Epoch: 3
INFO:root:Epoch: 4
INFO:root:Epoch: 5
INFO:root:Epoch: 6
INFO:root:Epoch: 7
INFO:root:Epoch: 8
INFO:root:Epoch: 9
INFO:root:Epoch: 10
INFO:root:Current common score: 1.1608847379240155
INFO:root:Try number: 2
INFO:root:Train with params: {'lr_all': 0.002, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.05}
INFO:root:Epoch: 1
INFO:root:Epoch: 2
INFO:root:Epoch: 3
INFO:root:Epoch: 4
INFO:root:Epoch: 5
INFO:root:Epoch: 6
INFO:root:Epoch: 7
INFO:root:Epoch: 8
INFO:root:Epoch: 9
INFO:root:Epoch: 10
INFO:root:Current common score: 1.1613874308488443
INFO:root:Try number: 3
INFO:root:Train with params: {'lr_all': 0.002, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.1}
INFO:root:Epoch: 1
INFO:root:Epoch: 2
INFO:root:Epoch: 3
INFO:root:Epoch: 4
INFO:root:Epoch: 5
INFO:root:Epoch: 6
INFO:root:Epoch: 7
INFO:root:Epoch: 8
IN

Best params: {'lr_all': 0.007, 'n_epochs': 50, 'n_factors': 40, 'reg_all': 0.2}
Best RMSE: 1.1113174596624342


# Model testing

In [19]:
metrics_calculator = SvdMetricsCalculator(test_matrix=test_matrix, model=best_svdpp_model,
                                          idx_to_user_id=user_mapping['idx_to_id'],
                                          idx_to_item_id=item_mapping['idx_to_id'])

metrics_calculator.calculate_rmse()

1.1789339324295245

In [20]:
# metrics_test_calculator = SvdTestMetricsCalculator(test_matrix=test_matrix, model=best_svdpp_model,
#                                                    idx_to_user_id=user_mapping['idx_to_id'],
#                                                    idx_to_item_id=item_mapping['idx_to_id'])
# 
# print("created")
# 
# metrics_test_calculator.calculate_unexpectedness()

# Model saving 

The following code saves the result object to reuse **the trained model** in the service

In [21]:
with open("./models/svd_pp.pkl", "wb") as f:
    pickle.dump(best_svdpp_model, f)